In [1]:
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime
import pandas as pd

In [5]:
# Spotify client credentials
client_id = "43964785e73c43698f52de9f8ea49068"
client_secret = "b5c8bf24d8454f50a9fdb8fa86f0d92c"

# Initialize Spotify client
credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials_manager)

In [6]:
# Define the Spotify playlist link
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbLiRSasKsNU9"

# Extract the playlist URI from the link
playlist_uri = playlist_link.split("/")[-1]

# Fetch playlist tracks using the Spotify API
data = sp.playlist_tracks(playlist_uri)

In [28]:
# Initialize an empty list to store album information
album_list = []

# Iterate over the 'items' in the 'data' object
for item in data['items']:
    # Extract album information from the nested structure within 'item'
    id = item['track']['album']['id']
    name = item['track']['album']['name']
    release_date = item['track']['album']['release_date']
    total_tracks = item['track']['album']['total_tracks']
    external_urls = item['track']['album']['external_urls']['spotify']

    # Create a dictionary 'album_element' to store the extracted information
    album_element = {
        'id': id,
        'name': name,
        'release_date': release_date,
        'total_tracks': total_tracks,
        'external_urls': external_urls
    }

    # Append 'album_element' to the 'album_list'
    album_list.append(album_element)

# 'album_list' now contains the extracted album information

In [29]:
# Initialize an empty list for artist information
artist_list = []

# Extract artist data from the 'data' object
for item in data['items']:
    for key, value in item.items():
        if key == 'track':
            for artist in value['artists']:
                # Create a dictionary with artist details
                artist_dict = {
                    'artist_id': artist['id'],
                    'artist_name': artist['name'],
                    'external_url': artist['href']
                }
                artist_list.append(artist_dict)

# 'artist_list' now contains artist information


In [30]:
# Initialize an empty list to store song information
song_list = []

# Loop through each item in the 'data' dictionary's 'items' list
for row in data['items']:
    # Extract various attributes of the song from the 'row' dictionary
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    
    # Create a dictionary representing a song element with extracted attributes
    song_element = {
        'song_id': song_id,
        'song_name': song_name,
        'song_duration': song_duration,
        'song_url': song_url,
        'song_popularity': song_popularity,
        'song_added': song_added,
        'album_id': album_id,
        'artist_id': artist_id
    }
    
    # Append the song element to the 'song_list'
    song_list.append(song_element)


In [35]:
# Create DataFrames from dictionaries and drop duplicates in a single line
album_df = pd.DataFrame.from_dict(album_list).drop_duplicates(subset=['id'])
song_df = pd.DataFrame.from_dict(song_list).drop_duplicates(subset=['song_id'])
artist_df = pd.DataFrame.from_dict(artist_list).drop_duplicates(subset=['artist_id'])

In [36]:
# Convert date columns to datetime objects
album_df['release_date'] = pd.to_datetime(album_df['release_date'])
song_df['song_added'] = pd.to_datetime(song_df['song_added'])